### Imports

In [1]:
import os
from utils.information_retrieval import run_commit_retrieval, create_folders, download_csv_files

### Inital Params

In [ ]:
commits_url = "https://api.github.com/repos/forecastingresearch/forecastbench-datasets/commits"
params = {'path': 'leaderboards/csv/leaderboard_overall.csv', 'per_page': 100}
headers = {'Authorization': f'token {os.getenv("GITHUB_API_KEY")}'}

### Information Retrieval Pipeline

In [ ]:
create_folders()

In [ ]:
run_commit_retrieval(commits_url, params, headers)

In [3]:
download_csv_files()